# Final Major Project Code

In [ ]:
import pathlib
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import ResNet152V2, InceptionV3, EfficientNetB4, DenseNet201
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
import scipy

In [ ]:
train_dir_cropped = 'RFMID - Cropped\\Train'
test_dir_cropped = 'RFMID - Cropped\\Test'
val_dir_cropped = 'RFMID - Cropped\\Validation'

img_height = 300
img_width = 300
batch_size = 32

In [ ]:
os.listdir()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir_cropped,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
)

validation_generator = test_datagen.flow_from_directory(
    val_dir_cropped,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir_cropped,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Model 1

base_model = InceptionV3(
    input_shape = (300,300,3),
    weights = "imagenet",
    include_top = False
)

for layer in base_model.layers[:10]:
  layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.4)(x)

predictions = Dense(7, activation='softmax')(x)
model1 = Model(inputs=base_model.inputs, outputs = predictions)

In [ ]:
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
with tf.device("/device:GPU:0"):
    history = model1.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )